In [174]:
import numpy as np
import statsmodels.formula.api as smf
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns  
from linares_plot import *

In [175]:
##Abrir los datos 
## Hacer un dataframe para cada subject en cada truco
## Concatenar esos dataframe (manteniendo información de truco i subject)
## Añadir también una columna con los tiempos (el que estaba y en segundo)

frames=[]
Trucos=['Truco 1', 'Truco 2', 'Truco 3', 'Truco 4', 'Truco 5', 'Truco 6', 'Truco 7', 'Truco 8' ]

for T in Trucos:
    Df=pd.read_excel(os.getcwd() + '\\ANOTACIONES_.xlsx', sheet_name= T )
    subj_idx = Df.columns[2:]
    #   
    for s in subj_idx:
        df_x = Df[['Tiempo_s', 'Tiempo', s]]
        df_x.columns=['time_s', 'time_min_sec', 'labels']
        df_x['subject']=s
        df_x['trick']=int(T.split(' ')[1])
        frames.append(df_x)

Df = pd.concat(frames, ignore_index=True)

C:\Users\David\Anaconda3\envs\python3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\David\Anaconda3\envs\python3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [176]:
### Quitar los momentos en que no hay eventos! (Nans fuera)
df_ = Df.loc[~pd.isna(df['labels'])]
df_.head()
len(df_) 

10302

In [177]:
## Como en ocasiones hay más de una acción en un mismo tiempo, hacer que en la columna label no haya comas
## En el caso que coocurran acciones, "multiplicar la linea", una para cada evento
## Como consecuencia, el len aumentará respecto df_

items=[]

for i in range(len(df_)):
    #print(i)
    n_actions = len(df_.labels.iloc[i].split(','))
    if n_actions==1:
        items.append([df_.time_s.iloc[i], df_.time_min_sec.iloc[i], 
                      df_.labels.iloc[i], df_.subject.iloc[i],  df_.trick.iloc[i]  ])
    elif n_actions>1:
        actions = df_.labels.iloc[i].split(',')
        for a in range(0, n_actions):
            items.append([df_.time_s.iloc[i], df_.time_min_sec.iloc[i], 
                          actions[a], df_.subject.iloc[i],  df_.trick.iloc[i]  ])
            
            
df_actions = pd.DataFrame(items)    
df_actions.columns = ['time_s', 'time_min_sec', 'labels', 'subject', 'trick']
print(len(df_actions))
df_actions.head(10)

14411


,time_s,time_min_sec,labels,subject,trick
0,8,00:00:08,Se,1,1
1,10,00:00:10,B,1,1
2,30,00:00:30,In,1,1
3,110,00:01:50,L,1,1
4,126,00:02:06,B,1,1
5,132,00:02:12,L,1,1
6,140,00:02:20,I,1,1
7,140,00:02:20,S,1,1
8,146,00:02:26,L,1,1
9,156,00:02:36,S,1,1


In [178]:
df_actions.labels.unique()

array(['Se', 'B', 'In', 'L', 'I', ' S', 'S', 'A', 'PERD.VIS', 'AA', 'R',
       'P', 'C', 'D', 'Si', ' R', 'M', 'F', ' F', 'A+', 'O', ' A',
       'PERD. VIS', 'A0', ' A+', 'P ', ' P', 'iN', ' L ', ' Si', 'PV',
       'N', 'a', 'perd.vis', ' C', ' D', ' B', ' Se', ' VP', 'VP', ' I',
       ' PV', ' L', 'H', 'DD', ' AA', 'T', ' a', ' N', ' O', 'AA+', 'se',
       'F ', 'S ', 'IN', 'PV ', 'SS', 'F  ', 'L ', ' I ', 'M  ', ' S   ',
       'N  ', 'sI', 'a?', 'N ', ' F ', ' In', 'VP ', ' L   ', 'S  ',
       ' A (ANTES)', 'TONTO BOCA P', 'L  ', ' A0', ' S ', 'C ', 'X',
       ' C ', '  L', ' R ', ' A ', 'F?', 'aa'], dtype=object)

In [179]:
### Correccion de fallos en las labels al entrar los datos
df_actions['labels'] = df_actions['labels'].replace([' Se'], ['Se'])
df_actions['labels'] = df_actions['labels'].replace(['se'], ['Se'])
df_actions['labels'] = df_actions['labels'].replace([' A'], ['A'])
df_actions['labels'] = df_actions['labels'].replace([' A '], ['A'])
df_actions['labels'] = df_actions['labels'].replace([' B'], ['B'])
df_actions['labels'] = df_actions['labels'].replace([' I'], ['I'])
df_actions['labels'] = df_actions['labels'].replace([' L'], ['L'])
df_actions['labels'] = df_actions['labels'].replace(['  L'], ['L'])
df_actions['labels'] = df_actions['labels'].replace(['L  '], ['L'])
df_actions['labels'] = df_actions['labels'].replace([' L '], ['L'])
df_actions['labels'] = df_actions['labels'].replace(['L '], ['L'])
df_actions['labels'] = df_actions['labels'].replace(['N '], ['N'])
df_actions['labels'] = df_actions['labels'].replace(['N  '], ['N'])
df_actions['labels'] = df_actions['labels'].replace([' N'], ['N'])
df_actions['labels'] = df_actions['labels'].replace([' S'], ['S'])
df_actions['labels'] = df_actions['labels'].replace([' S  '], ['S'])
df_actions['labels'] = df_actions['labels'].replace([' S   '], ['S'])
df_actions['labels'] = df_actions['labels'].replace(['S '], ['S'])
df_actions['labels'] = df_actions['labels'].replace([' S '], ['S'])
df_actions['labels'] = df_actions['labels'].replace(['S  '], ['S'])
df_actions['labels'] = df_actions['labels'].replace(['R '], ['R'])
df_actions['labels'] = df_actions['labels'].replace([' R'], ['R'])
df_actions['labels'] = df_actions['labels'].replace([' R '], ['R'])
df_actions['labels'] = df_actions['labels'].replace(['C '], ['C'])
df_actions['labels'] = df_actions['labels'].replace([' C '], ['C'])
df_actions['labels'] = df_actions['labels'].replace([' C'], ['C'])
df_actions['labels'] = df_actions['labels'].replace(['F '], ['F'])
df_actions['labels'] = df_actions['labels'].replace([' F '], ['F'])
df_actions['labels'] = df_actions['labels'].replace(['F  '], ['F'])
df_actions['labels'] = df_actions['labels'].replace([' F'], ['F'])
df_actions['labels'] = df_actions['labels'].replace(['M '], ['M'])
df_actions['labels'] = df_actions['labels'].replace(['M  '], ['M'])
df_actions['labels'] = df_actions['labels'].replace(['a '], ['a'])
df_actions['labels'] = df_actions['labels'].replace([' a'], ['a'])
df_actions['labels'] = df_actions['labels'].replace([' AA'], ['AA'])
df_actions['labels'] = df_actions['labels'].replace([' A0'], ['A0'])
df_actions['labels'] = df_actions['labels'].replace(['IN'], ['In'])
df_actions['labels'] = df_actions['labels'].replace([' In'], ['In'])
df_actions['labels'] = df_actions['labels'].replace(['iN'], ['In'])
df_actions['labels'] = df_actions['labels'].replace([' D'], ['D'])
df_actions['labels'] = df_actions['labels'].replace([' O'], ['O'])
df_actions['labels'] = df_actions['labels'].replace([' A+'], ['A+'])
df_actions['labels'] = df_actions['labels'].replace([' L   '], ['L'])
df_actions['labels'] = df_actions['labels'].replace([' I '], ['I'])
df_actions['labels'] = df_actions['labels'].replace(['PERD.VIS'], ['PV'])
df_actions['labels'] = df_actions['labels'].replace(['PERD. VIS'], ['PV'])
df_actions['labels'] = df_actions['labels'].replace(['perd.vis'], ['PV'])
df_actions['labels'] = df_actions['labels'].replace(['VP'], ['PV'])
df_actions['labels'] = df_actions['labels'].replace([' VP'], ['PV'])
df_actions['labels'] = df_actions['labels'].replace(['VP '], ['PV'])
df_actions['labels'] = df_actions['labels'].replace(['PV '], ['PV'])
df_actions['labels'] = df_actions['labels'].replace([' PV'], ['PV'])
df_actions['labels'] = df_actions['labels'].replace(['P '], ['P'])
df_actions['labels'] = df_actions['labels'].replace([' P'], ['P'])
df_actions['labels'] = df_actions['labels'].replace([' Si'], ['Si'])
df_actions['labels'] = df_actions['labels'].replace(['sI'], ['Si'])
df_actions['labels'] = df_actions['labels'].replace(['SS'], ['S'])
df_actions['labels'] = df_actions['labels'].replace(['DD'], ['D'])
df_actions['labels'] = df_actions['labels'].replace(['a?'], ['a'])
df_actions['labels'] = df_actions['labels'].replace(['F?'], ['F'])
df_actions['labels'] = df_actions['labels'].replace([' A (ANTES)'], ['A'])



df_actions.labels.unique()

array(['Se', 'B', 'In', 'L', 'I', 'S', 'A', 'PV', 'AA', 'R', 'P', 'C',
       'D', 'Si', 'M', 'F', 'A+', 'O', 'A0', 'N', 'a', 'H', 'T', 'AA+',
       'TONTO BOCA P', 'X', 'aa'], dtype=object)

#### Until clarification, I am excluding P, PV, VP, X, aa, H, T, AA+, TONTO BOCA P

In [183]:
df = df_actions.loc[(df_actions['labels']=='Se') | (df_actions['labels']=='B') | (df_actions['labels']=='In') | 
              (df_actions['labels']=='L') | (df_actions['labels']=='I') | (df_actions['labels']=='S') | 
              (df_actions['labels']=='A') | (df_actions['labels']=='AA') | (df_actions['labels']=='R') |
              (df_actions['labels']=='C') | (df_actions['labels']=='D') | (df_actions['labels']=='Si') |
              (df_actions['labels']=='M') | (df_actions['labels']=='F') | (df_actions['labels']=='A+') | 
              (df_actions['labels']=='O') | (df_actions['labels']=='A0') | (df_actions['labels']=='N') |
              (df_actions['labels']=='a')]

,time_s,time_min_sec,labels,subject,trick
0,8,00:00:08,Se,1,1
1,10,00:00:10,B,1,1
4,126,00:02:06,B,1,1
12,194,00:03:14,B,1,1
14,202,00:03:22,B,1,1
21,6,00:00:06,B,2,1
28,80,00:01:20,B,2,1
39,202,00:03:22,B,2,1
52,282,00:04:42,B,2,1
57,300,00:05:00,B,2,1
